# Fehlende Parlamentarier CH

Quelle: https://www.pragmatiqa.com/xodata/# und https://ws-old.parlament.ch/

In [4]:
import requests as r
import pandas as pd

Gesetzte Filter: Legislatur Periode = 51, Language = DE, Decision = NT (Nicht teilgenommen)

In [8]:
with open ("ch_parlamentarier.csv") as file:
    for line in file:
        data_ch = line.split("\n")
        print(data_ch)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [9]:
df = pd.read_csv('ch_parlamentarier.csv')

In [11]:
df.dtypes

FirstName         object
LastName          object
Canton            object
BusinessNumber     int64
BusinessTitle     object
VoteEnd           object
dtype: object

In [12]:
df.shape

(40634, 6)

In [18]:
df.sort_values('LastName').head(10)

,FirstName,LastName,Canton,BusinessNumber,BusinessTitle,VoteEnd
18937,Jean-Luc,Addor,VS,20203015,Bezahltes Unterschriftensammeln verbieten,2021-09-21T11:46:15
27862,Jean-Luc,Addor,VS,20210310,Aufnahme von Menschen aus Griechenland und Aus...,2022-06-09T17:14:41
33299,Jean-Luc,Addor,VS,20230016,AHVG. Änderung (Anpassung der Renten an die Te...,2023-03-01T09:53:18
35283,Jean-Luc,Addor,VS,1,Ordnungsantrag,2023-04-11T23:03:54
18918,Jean-Luc,Addor,VS,20194431,Betrug bei Unterschriftensammlungen soll im Na...,2021-09-21T11:45:27
35266,Jean-Luc,Addor,VS,1,Ordnungsantrag,2023-04-11T20:44:41
35250,Jean-Luc,Addor,VS,1,Ordnungsantrag,2023-04-11T20:42:04
26285,Jean-Luc,Addor,VS,20203447,Einrichtung einer alternativen Produktionskett...,2022-06-01T12:37:42
11429,Jean-Luc,Addor,VS,20203910,Deklaration des Produktionslandes von Brot und...,2021-03-03T17:18:47
39045,Jean-Luc,Addor,VS,20180077,Raumplanungsgesetz. Teilrevision. 2. Etappe,2023-06-15T15:17:23


Die Politiker aus welchem Kanton haben am meisten gefehlt?

In [32]:
fehlende_kanton = df.groupby('Canton')['BusinessNumber'].nunique().sort_values(ascending=False)
print(fehlende_kanton)

Canton
ZH    1741
BE    1571
VD    1191
AG     900
LU     875
GE     845
SO     733
SG     633
BL     600
TI     586
GR     542
VS     522
GL     520
FR     498
ZG     440
SZ     374
TG     296
BS     264
NE     254
NW     195
AI     180
SH     115
JU     108
OW      92
UR      51
AR       2
Name: BusinessNumber, dtype: int64


Welche Parlamentarier haben an wie vielen einzelnen Abstimmungen gefehlt?:

In [33]:
pd.set_option("display.max_rows", None)
verpasste_abstimmungen = df['LastName'].value_counts()
print(verpasste_abstimmungen)

Köppel                   1382
Landolt                  1157
Bäumle                   1015
Müller-Altermatt         1005
Hess                      847
Pfister                   734
Estermann                 728
Bertschy                  714
Kutter                    683
Nidegger                  675
Martullo-Blocher          647
Wermuth                   634
Badran                    540
Schneider                 503
von Siebenthal            498
Heer                      490
Moret                     475
Grossen                   456
Quadri                    428
Bellaiche                 419
Fluri                     415
Schneider-Schneiter       406
Buffat                    390
Mettler                   381
Fiala                     377
Gössi                     372
Wasserfallen              371
Markwalder                369
Lüscher                   366
Rösti                     353
Steinemann                353
Müller                    352
Maitre                    351
Nordmann  

In [34]:
round(verpasste_abstimmungen.mean(),2)

192.58

Die Liste zeigt alle verpassten Abstimmungen. Nun beinhalten gewisse Geschäfte mehrere Abstimmungen wie Ordnungsanträge oder Änderungsanträge. Fairer - und sinvnoller - wäre wohl also die Aussage, bei wie vielen Geschäften die Politiker mindestens eine Teilabstimmung verpasst haben.

In [24]:
verpasste_geschaefte = df.groupby('LastName')['BusinessNumber'].nunique().sort_values(ascending=False)
print(verpasste_geschaefte)

LastName
Köppel                   758
Bäumle                   535
Landolt                  520
Bertschy                 466
Hess                     454
Müller-Altermatt         430
Martullo-Blocher         390
Estermann                376
Wermuth                  355
Nidegger                 347
Pfister                  345
Grossen                  326
Badran                   308
Heer                     306
Kutter                   300
Bellaiche                284
Buffat                   252
Schneider-Schneiter      250
Moret                    246
Nordmann                 243
von Siebenthal           241
Siegenthaler             235
Quadri                   233
Lüscher                  226
Fiala                    223
Wasserfallen             223
Marti                    217
Rösti                    211
Mettler                  204
Markwalder               199
Schneider                197
Meyer                    196
Graber                   196
Giezendanner             195
Kelle

In [31]:
round(verpasste_geschaefte.mean(),2)

114.35

Nun gut, ein Geschäft kann man schnell mal verpassen, wenn man sich in der Wandelhalle rumtreibt. An wie vielen Tagen haben die Politiker gefehlt oder teilweise gefehlt? Ich habe dazu berechnet, an wie vielen Tagen die Politiker bei mindestens fünf verschiedenen Geschäften bei einer Abstimmung gefehlt haben.

In [26]:
df['VoteEnd'] = pd.to_datetime(df['VoteEnd']).dt.date
counts = df.groupby(['LastName', 'BusinessNumber','VoteEnd']).size().reset_index(name='Count')
filtered_counts = counts[counts['Count'] > 5]
gefehlte_tage = filtered_counts.groupby('LastName')['VoteEnd'].nunique().sort_values(ascending = False)
print(gefehlte_tage)

LastName
Landolt                  29
Köppel                   28
Pfister                  27
Müller-Altermatt         26
Bäumle                   24
Nidegger                 23
Hess                     19
Kutter                   15
Wermuth                  15
Estermann                15
von Siebenthal           14
Badran                   12
Schneider                11
Bertschy                 11
Müller                   10
Martullo-Blocher         10
Prezioso Batou           10
Moret                     9
Fluri                     9
Girod                     9
Quadri                    9
Buffat                    8
Gössi                     8
Lüscher                   8
Gafner                    7
Maitre                    7
Giezendanner              7
Markwalder                7
Keller                    7
Graber                    7
Fiala                     7
Grossen                   7
Steinemann                7
Heer                      7
Portmann                  7
Vogt       